In [19]:
import pandas as pd
import numpy as np
import webbrowser
import unicodedata
from updateDashboard import updateDashboard
from webscrapeData import updateData
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from createCharts import interactiveBubblePlot
from fuzzywuzzy import process

from createCharts import interactiveBubblePlot, gen_ppg_plot, gen_apg_plot, gen_rpg_plot, gen_fan_plot




updateData('2024-25')
data = pd.read_csv('../database/merged.csv')
player_dict = dict(zip(data['PLAYER_NAME'], data['PLAYER_ID']))

# Function to find the closest matching player
def get_player_id(user_input):
    best_match, score = process.extractOne(user_input, player_dict.keys())
    
    if score > 80:  # Set a threshold for similarity
        return player_dict[best_match]  # Return the corresponding ID
    else:
        return "No close match found."
def normalize_text(text):
    normalized_text = unicodedata.normalize('NFD', text)  # Decompose characters
    normalized_text = ''.join([c for c in normalized_text if not unicodedata.combining(c)])  # Remove accents
    return normalized_text.lower()

# Get data from csv file and create updated dataframe: converting each stat to per game stat

gen_ppg_plot()
gen_apg_plot()
gen_rpg_plot()
gen_fan_plot
data = pd.read_csv('../database/merged.csv')


data.rename(columns={'2024-25': 'SALARY'}, inplace=True)

data['PLAYER_NAME'] = data['PLAYER_NAME'].astype(str)
data['PTS'] = data['PTS'] / data['GP']
data['AST'] = data['AST'] / data['GP']
data['REB'] = data['REB'] / data['GP']
data['MIN'] = data['MIN'] / data['GP']
data['FGA'] = data['FGA'] / data['GP']
data['FGM'] = data['FGM'] / data['GP']
data['PLUS_MINUS'] = data['PLUS_MINUS'] / data['GP']
data['NORMALIZED_NAME'] = data['PLAYER_NAME'].apply(normalize_text)
data['NBA_FANTASY_PTS'] = data['NBA_FANTASY_PTS'] / data['GP']
data = data.dropna(subset=['SALARY'])
data['SALARY'] = data['SALARY'].replace('[\$,]', '', regex=True).str.strip()
data['SALARY'] = pd.to_numeric(data['SALARY'], errors='coerce')

selected_features = ['GP', 'AGE', 'PTS', 'AST', 'REB', 'MIN', 'PLUS_MINUS', 'FGM','FGA','W_PCT']
selected_features = ['GP', 'NBA_FANTASY_PTS', 'MIN', 'PLUS_MINUS', 'FGM','FGA','W_PCT']
X = data[selected_features]
y = data['SALARY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Standardizing Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training the Random Forest Regressor Model
rf_model = RandomForestRegressor(max_depth=None, min_samples_split=10, n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)


# Salary Prediction Function
def predict_salary_rf(model, scaler, stats):
    """
    Predict the salary of a player based on their stats.
    Args:
        model: Trained regression model.
        scaler: StandardScaler used for scaling features.
        stats: List of player stats in the same order as selected_features.
    Returns:
        Predicted salary as a float.
    """
    stats_scaled = scaler.transform([stats])  # Scale the input stats
    predicted_salary = model.predict(stats_scaled)  # Predict salary
    return predicted_salary[0]





updated Data in merged


In [20]:
import pandas as pd



data['Predicted_Salary'] = data.apply(
    lambda row: predict_salary_rf(
        rf_model,
        scaler,
        [row[feature] for feature in selected_features]
    ),
    axis=1
)


data['Predicted_Salary'] = data['Predicted_Salary'].astype(int)

C:\Users\amans\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

C:\Users\amans\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

C:\Users\amans\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

C:\Users\amans\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserW

In [ ]:
data.sort_values(by='Predicted_Salary', ascending=False, inplace = True)
data['Improved'] = data.apply(lambda x: x['Predicted_Salary'] > x['SALARY'], axis=1)
result_df = data[data['PLAYER_NAME'].str.endswith('Anunoby')]
result_df.head(10)



KeyError: "None of [Index(['FTPCT'], dtype='object')] are in the [columns]"

In [22]:
# Normalize both features
from sklearn.preprocessing import MinMaxScaler


# Step 1: Select features
mvp_features = data[['NBA_FANTASY_PTS', 'GP', 'W_PCT']].copy()

# Step 2: Normalize them
scaler = MinMaxScaler()
mvp_scaled = scaler.fit_transform(mvp_features)

# Step 3: Calculate MVP_SCORE using all three
# Let's say: 60% fantasy performance, 20% games played, 20% win percentage
data['MVP_SCORE'] = (
    0.6 * mvp_scaled[:, 0] + 
    0.2 * mvp_scaled[:, 1] + 
    0.2 * mvp_scaled[:, 2]
)

# Step 4: Sort and view
data.sort_values(by='MVP_SCORE', ascending=False, inplace=True)
data.head()



,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,SALARY,NORMALIZED_NAME,Predicted_Salary,MVP_SCORE
424,203999,Nikola Jokić,Nikola,1610612743,DEN,30.0,70,46,24,0.657,...,3,2,1,2,1,1,51415938,nikola jokic,41511687,0.901770
493,1628983,Shai Gilgeous-Alexander,Shai,1610612760,OKC,26.0,76,63,13,0.829,...,1,1,2,93,44,2,35859950,shai gilgeous-alexander,36153959,0.853883
180,203507,Giannis Antetokounmpo,Giannis,1610612749,MIL,30.0,67,40,27,0.597,...,4,28,3,5,2,4,48787676,giannis antetokounmpo,38501584,0.816796
263,1628369,Jayson Tatum,Jayson,1610612738,BOS,27.0,72,53,19,0.736,...,5,7,5,18,15,6,34848340,jayson tatum,38896870,0.768334
314,1626157,Karl-Anthony Towns,Karl-Anthony,1610612752,NYK,29.0,72,46,26,0.639,...,11,21,8,4,20,9,49205800,karl-anthony towns,41799493,0.736807
